# Library

In [1]:
from dotenv import load_dotenv
import os
from pymongo import MongoClient
from datetime import timedelta

# Connect to the server

In [2]:
load_dotenv()

uri = os.getenv("MONGODB_URI")
client = MongoClient(uri)
client.admin.command("ping")
print("Connected OK!")

Connected OK!


In [3]:
db = client[os.getenv("DB_NAME")]
coll = db[os.getenv("COLL_NAME")]

# Some simple queries

In [4]:
doc = coll.find_one()
doc

{'_id': ObjectId('680ac83630fef610e321edb3'),
 'game': 'mhs',
 'player_id': 'test_player1',
 'timestamp': datetime.datetime(2025, 4, 24, 23, 24, 38, 277000),
 'event_type': 'test_event1',
 'event_data': {'enemy_type': 'bug', 'damage': 100.0},
 'device_info': {'platform': 'WebGL', 'browser': 'Chrome'}}

In [5]:
cursor = coll.find(
    {"player_id": "wenyi3@mhs", "event_type": "DialogueEvent", 
     "$and":[{"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}}]}
)

In [6]:
list(cursor)[:10]

[{'_id': ObjectId('688392bf72ebf4f11406847a'),
  'game': 'mhs',
  'player_id': 'wenyi3@mhs',
  'timestamp': datetime.datetime(2025, 7, 25, 14, 20, 47, 649000),
  'event_type': 'DialogueEvent',
  'event_data': '{"dialogueEventType":"DialogueNodeEvent","conversationId":"30","nodeId":"95"}',
  'device_info': {'platform': 'WebGLPlayer', 'browser': 'WebGL_Browser'}},
 {'_id': ObjectId('6883953a72ebf4f114068508'),
  'game': 'mhs',
  'player_id': 'wenyi3@mhs',
  'timestamp': datetime.datetime(2025, 7, 25, 14, 31, 22, 380000),
  'event_type': 'DialogueEvent',
  'event_data': '{"dialogueEventType":"DialogueNodeEvent","conversationId":"31","nodeId":"72"}',
  'device_info': {'platform': 'WebGLPlayer', 'browser': 'WebGL_Browser'}},
 {'_id': ObjectId('6883953b72ebf4f11406850a'),
  'game': 'mhs',
  'player_id': 'wenyi3@mhs',
  'timestamp': datetime.datetime(2025, 7, 25, 14, 31, 23, 47000),
  'event_type': 'DialogueEvent',
  'event_data': '{"dialogueEventType":"DialogueNodeEvent","conversationId":"31

# U1.C1

During this game progress, followed by the instruction of DANI, players set up their virtual look, investigated their lock-up, and also followed the waypoint to talk to Dr. Toppo. Completing the progress after Dr. Toppo let the players talk to other team members. White color means they didn't complete this progress point; Green means they completed this progress.

In [8]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs",
]

In [7]:
def color_u1c1_for_pid(coll, pid):
    
    has_q3129 = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"31"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"29"'}}
        ]
    }) > 0

    if not has_q3129:
        return "white"
    else:
        return "green"

In [9]:
pids_u1c1_to_color = {pid: color_u1c1_for_pid(coll, pid) for pid in pids}
pids_u1c1_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'green', 'sean1@mhs': 'white'}

# U1.C2

During this progress point, players start by following the waypoints to talk to each team member, collecting information about scientific argumentation, getting to know each team member, and end with going back to Dr. Toppo. White represents not completing this progress; Green means completing this progress point. 

In [10]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs",
]

In [11]:
def color_u1c2_for_pid(coll, pid):
    
    has_q3098 = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"30"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"98"'}}
        ]
    }) > 0

    if not has_q3098:
        return "white"
    else:
        return "green"

In [12]:
pids_u1c2_to_color = {pid: color_u1c1_for_pid(coll, pid) for pid in pids}
pids_u1c2_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'green', 'sean1@mhs': 'white'}

# U1.C3

During this progress point, players talked with Dr. Toppo and watched her argumentation tutorials and completed two sections of the argumentation. White means not completing this progress; Green means submitting the correct argument at the first attempt; Yellow means submitting the correct argument using more than one attempt.

In [253]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs",
]

In [254]:
def color_u1c3_for_pid(coll, pid):
    
    has_q34 = coll.count_documents({
        "player_id": pid,
        "event_type": "questEvent",
        "$and": [
            {"event_data": {"$regex": r'"questEventType"\s*:\s*"questActiveEvent"'}},
            {"event_data": {"$regex": r'"questID"\s*:\s*"34"'}}
        ]
    }) > 0

    if not has_q34:
        return "white"

    has_node_33_or_25 = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"70"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"(33|25)"'}}
        ]
    }) > 0

    return "yellow" if has_node_33_or_25 else "green"

In [255]:
pids_u1c3_to_color = {pid: color_u1c3_for_pid(coll, pid) for pid in pids}
pids_u1c3_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'yellow', 'sean1@mhs': 'white'}

# U1.C4

During this progress point, players completed the conversation with Dr. Toppo, encountered a crash of the spaceship, and then escaped the ship along with other team members using the escape pod. White means not completing the progress, while green means completing the progress.

In [13]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs",
]

In [14]:
def color_u1c4_for_pid(coll, pid):
    
    has_q34 = coll.count_documents({
        "player_id": pid,
        "event_type": "questEvent",
        "$and": [
            {"event_data": {"$regex": r'"questEventType"\s*:\s*"questFinishEvent"'}},
            {"event_data": {"$regex": r'"questID"\s*:\s*"34"'}}
        ]
    }) > 0

    if not has_q34:
        return "white"
    else:
        return "green"

In [15]:
pids_u1c4_to_color = {pid: color_u1c4_for_pid(coll, pid) for pid in pids}
pids_u1c4_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'green', 'sean1@mhs': 'white'}

# U2.C1

During this progress point, players need to match topographic glyphs based on the 2D shape of the topographic geography from the top view. The color is white if the player doesn't complete the progress; The color is yellow if they triggered the dialogue feedbacks, representing they have more than 4 attempts to make it right or seeking help from DANI. The color is green if they match the glyphs within 4 attempts without help from DANI. After this task, players will have a better idea of the knowledge of topographic knowledge. At the end, players will find a way to exit the temple.     

In [256]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs"
]

In [257]:
def color_u2c1_for_pid(coll, pid):
    
    has_finish21 = coll.count_documents({
    "player_id": pid,
    "event_type": "questEvent",
    "$and": [
        {"event_data": {"$regex": r'"questEventType"\s*:\s*"questFinishEvent"'}},
        {"event_data": {"$regex": r'"questID"\s*:\s*"21"'}}
    ]
    }) > 0

    if not has_finish21:
        return "white"

    else:
    
        has_bad_nodes = coll.count_documents({
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"68"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"(23|28|27|31)"'}}
            ]
        }) > 0

        has_node_29 = coll.count_documents({
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"68"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"29"'}}
            ]
        }) > 0

        if has_node_29 and not has_bad_nodes:
            return "green"
        else:
            return "yellow"

In [258]:
pids_u2c1_to_color = {pid: color_u2c1_for_pid(coll, pid) for pid in pids}
pids_u2c1_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'green', 'sean1@mhs': 'white'}

# U2.C2

During this progress point, players will meet Anderson and help her fix the hoverboard. After successfully fixing it, players can navigate with the hoverboard. Then, players need to place a waypoint on the topographic map based on the location description of Dr. Toppo. Following the placed waypoint, players need to find where Dr. Toppo is. White if the player didn't complete the progress, yellow if the timeduration of finding Dr. Toppo is larger than 5 minutes, otherwise it's green.

In [259]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs"
]

In [260]:
FIVE_MIN = timedelta(minutes=5)

In [261]:
def color_u2c2_for_pid(coll, pid):
    
    has_complete = coll.count_documents({
    "player_id": pid,
    "event_type": "DialogueEvent",
    "$and": [
        {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"20"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"26"'}},
    ]
    }) > 0

    if not has_complete:
        return "white"

    else:
    
        e_201 = coll.find_one(
        {
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
                {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"20"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"1"'}},
            ],
        },
        sort=[("timestamp", 1)],
        projection={"_id": 0, "timestamp": 1},
        )

        e_1946 = coll.find_one(
        {
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
                {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"19"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"46"'}},
            ],
        },
        sort=[("timestamp", 1)],
        projection={"_id": 0, "timestamp": 1},
        )

        if not e_201 or not e_1946:
            return "yellow"

        delta = abs(e_201["timestamp"] - e_1946["timestamp"])
        return "yellow" if delta > FIVE_MIN else "green"

In [262]:
pids_u2c2_to_color = {pid: color_u2c2_for_pid(coll, pid) for pid in pids}
pids_u2c2_to_color

{'sean3@mhs': 'yellow', 'wenyi3@mhs': 'green', 'sean1@mhs': 'white'}

# U2.C3

Similar to the last progress point, players need to place the waypoints on the topographic map and find the other two team members based on the descriptions. If the player finds both team members using timedurations less than 5 minutes, then the color is green; otherwise, it's yellow.  

In [263]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs"
]

In [264]:
FIVE_MIN = timedelta(minutes=5)

In [265]:
def color_u2c3_for_pid(coll, pid):
    
    has_complete = coll.count_documents({
    "player_id": pid,
    "event_type": "DialogueEvent",
    "$and": [
        {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"22"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"18"'}},
    ]
    }) > 0

    if not has_complete:
        return "white"

    else:
    
        s_01 = coll.find_one(
        {
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
                {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"20"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"33"'}},
            ],
        },
        sort=[("timestamp", 1)],
        projection={"_id": 0, "timestamp": 1},
        )

        e_01 = coll.find_one(
        {
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
                {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"21"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"1"'}},
            ],
        },
        sort=[("timestamp", 1)],
        projection={"_id": 0, "timestamp": 1},
        )

        s_02 = coll.find_one(
        {
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
                {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"21"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"15"'}},
            ],
        },
        sort=[("timestamp", 1)],
        projection={"_id": 0, "timestamp": 1},
        )

        e_02 = coll.find_one(
        {
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
                {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"22"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"1"'}},
            ],
        },
        sort=[("timestamp", 1)],
        projection={"_id": 0, "timestamp": 1},
        )

        if not e_01 or not e_02:
            return "yellow"

        delta_1 = abs(e_01["timestamp"] - s_01["timestamp"])

        delta_2 = abs(e_02["timestamp"] - s_02["timestamp"])

        if delta_1 <= FIVE_MIN and delta_2 <= FIVE_MIN:
            return "green"
        else:
            return "yellow"

In [266]:
pids_u2c3_to_color = {pid: color_u2c3_for_pid(coll, pid) for pid in pids}
pids_u2c3_to_color

{'sean3@mhs': 'yellow', 'wenyi3@mhs': 'green', 'sean1@mhs': 'white'}

# U2.C4

In this progress point, players need to place the glyphs on the wall based on the humidity level. The color will represent how many attempts they used to arrange the glyphs correctly. If players arrange the glyphs correctly within five attempts, and place glyphs by themselves, the color will be green; otherwise, it will be yellow.

In [231]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs"
]

In [232]:
def color_u2c4_for_pid(coll, pid):
    
    has_finish_24 = coll.count_documents({
    "player_id": pid,
    "event_type": "DialogueEvent",
    "$and": [
        {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"23"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"17"'}},
    ]
    }) > 0

    if not has_finish_24:
        return "white"

    else:
    
        has_bad_nodes = coll.count_documents({
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"74"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"(16|17|20|22)"'}}
            ]
        }) > 0

        has_node_21 = coll.count_documents({
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"74"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"21"'}}
            ]
        }) > 0

        if has_node_21 and not has_bad_nodes:
            return "green"
        else:
            return "yellow"

In [233]:
pids_u2c4_to_color = {pid: color_u2c4_for_pid(coll, pid) for pid in pids}
pids_u2c4_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'green', 'sean1@mhs': 'white'}

# U2.C5

In this progress point, players need to figure out the correct argumentation component based on the text descriptions. Once the player chooses the correct one, positive feedback will be given; otherwise, a negative one will trigger. By calculating the score of summing positive and negative points together, we can know the color of the corresponding block. Yellow if the score is less than four; Green if the score is equal to or larger than four; White if the player didn't complete the progress point.

In [234]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs"
]

In [235]:
POS_NODES = [140,142,143,146,147,148,165,166,167,168,169,170,172,173,174,175,177,178,179,180,181,182,183,184,185,186]
NEG_NODES = [137,144,145,187,188,189,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211]

pos_alt = "|".join(str(n) for n in POS_NODES)
neg_alt = "|".join(str(n) for n in NEG_NODES)

In [236]:
def color_u2c5_for_pid(coll, pid):
    
    has_23_42 = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"23"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"42"'}}
        ]
    }) > 0

    if not has_23_42:
        return "white"

    else:
        pos_count = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"26"'}},
            {"event_data": {"$regex": rf'"nodeId"\s*:\s*"(?:{pos_alt})"'}}
        ]
        })

        neg_count = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"26"'}},
            {"event_data": {"$regex": rf'"nodeId"\s*:\s*"(?:{neg_alt})"'}}
        ]
        })

        score = pos_count - (neg_count / 3.0)

        if score < 4:
            return "yellow"
        else:
            return "green"

In [237]:
pids_u2c5_to_color = {pid: color_u2c5_for_pid(coll, pid) for pid in pids}
pids_u2c5_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'green', 'sean1@mhs': 'white'}

# U2.C6

In this progress point, Dr. Toppo will ask players what is the key factors that determine the watershed size, if the player chose the "flow rate", then the color is green, other choices will lead to yellow. If the players didn't complete this progress then the color will be white.

In [238]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs"
]

In [239]:
def color_u2c6_for_pid(coll, pid):
    
    has_finish18284 = coll.count_documents({
    "player_id": pid,
    "event_type": "DialogueEvent",
    "$and": [
        {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
                {"event_data": {"$regex": r'"conversationId"\s*:\s*"18"'}},
                {"event_data": {"$regex": r'"nodeId"\s*:\s*"284"'}},
    ]
    }) > 0

    if not has_finish18284:
        return "white"

    else:
    
        has_bad_nodes = coll.count_documents({
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"20"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"(44|45)"'}}
            ]
        }) > 0

        has_good_node = coll.count_documents({
            "player_id": pid,
            "event_type": "DialogueEvent",
            "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"20"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"43"'}}
            ]
        }) > 0

        if has_bad_nodes:
            return "yellow"
        else:
            return "green"

In [240]:
pids_u2c6_to_color = {pid: color_u2c6_for_pid(coll, pid) for pid in pids}
pids_u2c6_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'green', 'sean1@mhs': 'white'}

# U2.C7

During this progress point, players will complete the scientific argumentation of this unit. For relevant feedback dialogue of this quest, only one feedback is the positive one (if the argumentation is correct), others are negative ones. If the player got a positve feedback and the negative feedback count is less or equal to 3 then the color is green, otherwise the color is yellow. If the players didn't coplete the progress, then the color is white.

In [241]:
pids = [
    "sean3@mhs",
    "wenyi3@mhs",
    "sean1@mhs"
]

In [242]:
NEG_NODES = [11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30]
neg_alt = "|".join(str(n) for n in NEG_NODES)

In [243]:
def color_u2c7_for_pid(coll, pid):
    
    has_20_75 = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"20"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"(74|75)"'}}
        ]
    }) > 0

    if not has_20_75:
        return "white"

    else:
        has_complete = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"27"'}},
            {"event_data": {"$regex": r'"nodeId"\s*:\s*"7"'}}
        ]
        })

        neg_count = coll.count_documents({
        "player_id": pid,
        "event_type": "DialogueEvent",
        "$and": [
            {"event_data": {"$regex": r'"dialogueEventType"\s*:\s*"DialogueNodeEvent"'}},
            {"event_data": {"$regex": r'"conversationId"\s*:\s*"27"'}},
            {"event_data": {"$regex": rf'"nodeId"\s*:\s*"(?:{neg_alt})"'}}
        ]
        })

        if has_complete and neg_count <= 3:
            return "green"
        else:
            return "yellow"

In [244]:
pids_u2c7_to_color = {pid: color_u2c7_for_pid(coll, pid) for pid in pids}
pids_u2c7_to_color

{'sean3@mhs': 'green', 'wenyi3@mhs': 'yellow', 'sean1@mhs': 'white'}